In [2]:
import cv2
cap = cv2.VideoCapture(0)
import os
import numpy as np
import joblib
import threading 
loaded = joblib.load("svm_model/all_loaded_models")
scaler = loaded.get('scaler')
grid_search = loaded.get('grid_search')
# os.makedirs("color_face",exist_ok=True)
# os.makedirs("gray_face",exist_ok=True)


from deepface import DeepFace
frame_count = 0
# from pinecone import Pinecone
# model = DeepFace.build_model("Facenet")
# pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API"),environment = "us-east-1")
# vector_index = pinecone_client.Index("images-kb")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
while True:
    ret,frame = cap.read()
    if not ret:
        print("Failed to grab the frame")
        break
    frame_count+=1
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,   # Image size reduction
        minNeighbors=7,    # How many neighbors for a candidate rectangle
        minSize=(80, 80)   # Minimum face size
    )

    # Draw rectangles around faces
  
    
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        prediction = "unknown"
        if frame_count % 5 == 0:
            try:
                embedding = DeepFace.represent(
                            img_path=face_img,
                            model_name="ArcFace",  # or Facenet, VGG-Face, etc.
                            detector_backend="retinaface",  # best for faces
                            enforce_detection=False
                        )
                print("Embedding created ✅")
                emb_obj = embedding[0]['embedding']
                embeddings = np.array(emb_obj).reshape(1,-1)
                scaled_emb = scaler.transform(embeddings)
                prediction =  grid_search.best_estimator_.predict(scaled_emb)[0]

            except Exception as e:
                print("error",e)
                

        # Display name on frame
        cv2.putText(frame, prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imshow("camera",frame)
    key = cv2.waitKey(20) & 0xFF 

    # if key == ord('s'):
    #     for (x,y,w,h) in faces:
    #         # save graysacle face
            
    #         image = gray[y:y+h,x:x+w]
    #         file_name = f"gray_face/face_{gray_count}.png"
    #         cv2.imwrite(file_name,image)
    #         gray_count +=1 
    #         # save color_image face
    #         color_file_name = f"color_face/color_face{color_count}.png"
    #         color_image =frame[y:y+h,x:x+w]
    #         cv2.imwrite(color_file_name,color_image)
    #         color_count+=1

    #         print("image saved successfully")

    if key == ord("q"):
    
        break
cap.release()
cv2.destroyAllWindows()

Embedding created ✅
Embedding created ✅
Embedding created ✅


: 

In [ ]:
print(loaded)

{'scaler': StandardScaler(), 'grid_search': GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             verbose=2)}


: 